In [10]:
import sys
import itertools

import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import savgol_filter
from scipy.stats import zscore

from tqdm import tqdm

import neuroseries as nts


import time
import matplotlib.pyplot as plt
# %matplotlib qt

import seaborn as sns 
import bk.load
import bk.compute
import bk.plot

import os
%load_ext memory_profiler
import cProfile

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [11]:
import basefunction.vBaseFunctions3 as vbf

In [12]:
def wavelet_spectrogram(lfp,fmin,fmax,nfreq):
    f_wv = pow(2,np.linspace(np.log2(fmin),np.log2(fmax),nfreq))
    output = vbf.wvSpect(lfp_state.values,f_wv)
    
    return f_wv,lfp.as_units('s').index.values,output

In [13]:
import matplotlib as mpl
COLOR = 'white'
# mpl.rcParams['text.color'] = COLOR
# mpl.rcParams['axes.labelcolor'] = COLOR
# mpl.rcParams['axes.edgecolor'] = COLOR
# mpl.rcParams['xtick.color'] = COLOR
# mpl.rcParams['ytick.color'] = COLOR
# mpl.rcParams['figure.facecolor'] = '#282C34'
# mpl.rcParams['axes.facecolor'] = '#282C34'
mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['font.family'] = 'Sawasdee'
# %matplotlib qt

In [14]:
def plot_data(lfp,neurons,metadata,region,window):
    t_start = window[0]
    t_end = window[1]
    intervals = nts.IntervalSet(t_start,t_end,time_units='s')
    
    ripples = bk.load.ripples()
    
    t_pop,p = population_rates(neurons[(metadata.Region == 'BLA') & (metadata.Type == 'Int')],0.01,t_start,t_end)
    lfp_state = lfp.restrict(intervals)
    
    
#     f,t,Sxx = scipy.signal.spectrogram(lfp_state.as_units('s')[t_start:t_end].values,1250,nperseg=250,noverlap=125)
    
    f_wv = pow(2,np.linspace(np.log2(1),np.log2(100),200))

    output = vbf.wvSpect(lfp_state.values,f_wv)
    fig,ax = plt.subplot_mosaic('''A
                                   B
                                   B
                                   C
                                   C
                                ''',figsize = [12,8])
    
    ax['A'].plot(lfp_state.as_units('s')[t_start:t_end])
    ax['A'].set_ylim([-1000, 1000])
    ax['A'].set_xlim([t_start,t_end])
    
    plt.sca(ax['B'])
#     plt.pcolormesh(t+t_start, f, Sxx, shading='auto')
    ax['B'].pcolormesh(lfp_state.as_units('s').index,f_wv,output[0],shading = 'gouraud',vmin = 0,vmax = 500,rasterized = True)

    plt.xlim([t_start,t_end])
    plt.ylim([0,100])
#     plt.clim([0, 5000])
    
    plt.sca(ax['C'])
#     bk.plot.rasterPlot(neurons[(metadata.Region == 'BLA') & (metadata.Type == 'Int')],[t_start,t_end],width=1)
    for i,n in enumerate(neurons[(metadata.Region == 'BLA') & (metadata.Type == 'Pyr')]):
        t = n.restrict(intervals).as_units('s').index
        plt.plot(t,t-t + i,'k|',rasterized = True)

    for i,n in enumerate(neurons[(metadata.Region == 'BLA') & (metadata.Type == 'Int')],len(neurons[(metadata.Region == 'BLA') & (metadata.Type == 'Pyr')])):
        t = n.restrict(intervals).as_units('s').index
        plt.plot(t,t-t + i,'r|',rasterized = True)
                         
#     for i in ripples['peak'].as_units('s').index:
#         plt.axvline(i,color = 'red')
    plt.bar(t_pop,p*5,t_pop[1]-t_pop[0],alpha = 0.5)
    plt.xlim([t_start,t_end])
    
    
#     plt.suptitle(f'SWS Sleep BLA Shank - 16 Channel - {chan}')
    ax['A'].sharex(ax['C'])
    plt.tight_layout()

In [15]:
def population_rates(neurons,binSize = 0.010,start=0,end=None):
    t,binned = bk.compute.binSpikes(neurons,binSize,start,end)
    pop = binned.sum(0)
    return t,pop

In [9]:
bk.load.current_session_linux()
states = bk.load.states()
chan = bk.load.random_channel('BLA')
lfp = bk.load.lfp(chan,0,5000)
neurons,metadata = bk.load.loadSpikeData(bk.load.path)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /home/billel/Data/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [16]:
%matplotlib qt
plot_data(lfp,neurons,metadata,'BLA',[1927,1950])
plt.suptitle(f'SWS Sleep BLA Shank - 16 Channel - {chan}')
plt.tight_layout()

You are using 0.01 ms bins with the function fast off. Consider using "Fast = True" in order to speed up the computations


/home/billel/anaconda3/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py:238: RuntimeWarning: Glyph 8722 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/home/billel/anaconda3/lib/python3.7/site-packages/matplotlib/backends/backend_agg.py:201: RuntimeWarning: Glyph 8722 missing from current font.
  font.set_text(s, 0, flags=flags)


In [7]:
for s,intervals in states.items():
    intervals = intervals.drop_short_intervals(5)

    t_start = intervals.as_units('s').iloc[0][0]+35
    t_end = t_start + 5
    t_pop,p = population_rates(neurons[(metadata.Region == 'BLA') & (metadata.Type == 'Int')],0.025,t_start,t_end)
    lfp_state = lfp.restrict(intervals)
    
    
    f,t,Sxx = scipy.signal.spectrogram(lfp_state.as_units('s')[t_start:t_end].values,1250,nperseg=250,noverlap=125)
    fig,ax = plt.subplot_mosaic('''A
                                   B
                                   C
                                   C
                                   C
                                ''',figsize = [12,8])
    ax['A'].plot(lfp_state.as_units('s')[t_start:t_end])
    ax['A'].set_ylim([-1000, 1000])
    ax['A'].set_xlim([t_start,t_end])
    
    plt.sca(ax['B'])
    plt.pcolormesh(t+t_start, f, Sxx, shading='auto')
    plt.xlim([t_start,t_end])
    plt.ylim([0,100])
    plt.clim([0, 5000])
    
    plt.sca(ax['C'])
    bk.plot.rasterPlot(neurons[metadata.Region == 'BLA'],[t_start,t_end],width=1)
    plt.bar(t_pop,p,t_pop[1]-t_pop[0])
    plt.xlim([t_start,t_end])
    
    
    plt.suptitle(s + ' BLA Shank - 16 Channel - 123')
    plt.tight_layout()
#     plt.savefig(f'/mnt/work/Figures Gabrielle/{s}-BLA-S16C123.png',dpi = 300)

NameError: name 'states' is not defined